## ESTADISTICA DESCRIPTIVA

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import statistics as st
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def f_leer_archivo(path,sheet):
    return pd.read_excel(path,sheet)
mtf1 = f_leer_archivo('MyST_LAB2_FMHL.xlsx','Historico MT5')
mtm1 = f_leer_archivo('MyST_LAB2_MMM.xlsx','Historico MT5')
mtp1 = f_leer_archivo('MyST_LAB2_PHMC.xlsx','Historico MT5')
mta1 = f_leer_archivo('MyST_LAB2_ARG.xlsx','Historico MT5')

In [3]:
def f_columnas_tiempos(data):
    col1=pd.to_datetime(data.iloc[:,0])
    col2=pd.to_datetime(data.iloc[:,8])
    data['Time_change']=col2-col1 
    for i in range(len(data)):
        data['Time_change'][i] = data['Time_change'][i].total_seconds() 
    return data
mtf = f_columnas_tiempos(mtf1)
mtm = f_columnas_tiempos(mtm1)
mtp = f_columnas_tiempos(mtp1)
mta = f_columnas_tiempos(mta1)

In [4]:
def f_pip_size(ticker):
    diccionario={'EURUSD':0.00001, 'GBPUSD':0.00001,'EURGBP':0.00001,'USDCAD':0.00001,'USDCHF':0.00001,
                 'GBPJPY':0.001,'USDJPY':0.001,'EURCAD':0.001,'USDCNH':0.00001,'GBPNZD':0.00001,'AUDUSD':0.00001,
                'USDMXN':0.00001,'CADCHF':0.00001,'CADJPY':0.001,'USDNOK':0.00001,'USDSGD':0.00001,'USDPLN':0.00001,
                'USDTRY':0.00001,'EURJPY':0.001,'EURMXN':0.00001,'NZDUSD':0.00001,'GBPCHF':0.00001,'AUDCHF':0.00001,
                'GBPAUD':0.00001}
    return (round(1/diccionario[ticker]))
f_pip_size('EURUSD')

100000

In [5]:
def f_columnas_pips(data):
    for i in range(len(data)):
        if data.loc[i,'Type']=='sell':
            data.loc[i,'pips']=(data.iloc[i,5]-data.iloc[i,9])*f_pip_size(data.loc[i,'Symbol'])
        else:
            data.loc[i,'pips']=(data.iloc[i,9]-data.iloc[i,5])*f_pip_size(data.loc[i,'Symbol'])
    data['pips_acm']=data['pips'].cumsum()
    data['profit_acm']=data['Profit'].astype(float).cumsum()
    return data
col_pipsf = f_columnas_pips(mtf)
col_pipsm = f_columnas_pips(mtp)
col_pipsp = f_columnas_pips(mtm)
col_pipsa = f_columnas_pips(mta)

In [6]:
def f_estadisticas_ba(data,df):
    df_1=pd.DataFrame({'medida':['Ops totales','Ganadoras','Ganadoras_c','Ganadoras_v','Perdedoras','Perdedoras_c',
                             'Perdedoras_v','Mediana (profit)','Mediana (pips)','r_efectividad','r_proporcion',
                             'r_efectividad_c', 'r_efectividad_v'],
                   'valor':np.zeros(13),
                  'descripcion':['Operaciones totales','Operaciones ganadoras','Operaciones ganadoras de compra',
                                'Operaciones ganadoras de venta','Operaciones perdedoras','Operaciones perdedoras de compra',
                                'Operaciones perdedoras de venta','Mediana de profit de operaciones',
                                 'Mediana de pips de operaciones','Ganadoras totales/ Operaciones totales',
                                'Ganadoras totales/ Perdedoras totales','Ganadoras compras/ Operaciones totales',
                                 'Ganadoras ventas/ Operaciones totales']})
    df_1.iloc[0,1]=len(data)
    df_1.iloc[1,1]=len(data[data['Profit']>0])
    df_1.iloc[2,1]=len(data[(data['Profit']>0) & (data['Type']=='buy')])
    df_1.iloc[3,1]=len(data[(data['Profit']>0) & (data['Type']=='sell')])
    df_1.iloc[4,1]=len(data[data['Profit']<0])
    df_1.iloc[5,1]=len(data[(data['Profit']<0) & (data['Type']=='buy')])
    df_1.iloc[6,1]=len(data[(data['Profit']<0) & (data['Type']=='sell')])
    df_1.iloc[7,1]=data['Profit'].median()
    df_1.iloc[8,1]=data['pips'].median()
    df_1.iloc[9,1]=df_1.iloc[1,1]/df_1.iloc[0,1]
    df_1.iloc[10,1]=df_1.iloc[1,1]/df_1.iloc[5,1]
    df_1.iloc[11,1]=df_1.iloc[3,1]/df_1.iloc[1,1]
    df_1.iloc[10,1]=df_1.iloc[4,1]/df_1.iloc[1,1]
    df_1['valor']=round(df_1['valor'],2)
    
    uniques=data['Symbol'].unique()
    rank=[]
    for i in uniques:
        rank.append(round(len(data[(data['Profit']>0) & (data['Symbol']==i)])/len(data[data['Symbol']==i])*100,2))
    df_2_ranking=pd.DataFrame({'Symbol':uniques,'rank %':rank})
    
    diccionario={'tabla':df_1,'ranking':df_2_ranking}
    return diccionario[df]

In [7]:
f_estadisticas_ba(mtf,'tabla')

,medida,valor,descripcion
0,Ops totales,23.00,Operaciones totales
1,Ganadoras,10.00,Operaciones ganadoras
2,Ganadoras_c,7.00,Operaciones ganadoras de compra
3,Ganadoras_v,3.00,Operaciones ganadoras de venta
4,Perdedoras,13.00,Operaciones perdedoras
5,Perdedoras_c,2.00,Operaciones perdedoras de compra
6,Perdedoras_v,11.00,Operaciones perdedoras de venta
7,Mediana (profit),-56.00,Mediana de profit de operaciones
8,Mediana (pips),-53.00,Mediana de pips de operaciones
9,r_efectividad,0.43,Ganadoras totales/ Operaciones totales


In [8]:
f_estadisticas_ba(mtf,'ranking')

,Symbol,rank %
0,USDCAD,50.00
1,EURUSD,66.67
2,EURGBP,0.00
3,GBPUSD,16.67
4,USDJPY,100.00
5,GBPJPY,66.67
6,USDCHF,50.00


In [9]:
f_estadisticas_ba(mtm,'tabla')

,medida,valor,descripcion
0,Ops totales,22.00,Operaciones totales
1,Ganadoras,11.00,Operaciones ganadoras
2,Ganadoras_c,9.00,Operaciones ganadoras de compra
3,Ganadoras_v,2.00,Operaciones ganadoras de venta
4,Perdedoras,11.00,Operaciones perdedoras
5,Perdedoras_c,9.00,Operaciones perdedoras de compra
6,Perdedoras_v,2.00,Operaciones perdedoras de venta
7,Mediana (profit),-31.23,Mediana de profit de operaciones
8,Mediana (pips),-14.50,Mediana de pips de operaciones
9,r_efectividad,0.50,Ganadoras totales/ Operaciones totales


In [10]:
f_estadisticas_ba(mtm,'ranking')

,Symbol,rank %
0,USDJPY,50.0
1,USDCHF,80.0
2,USDCNH,0.0
3,USDMXN,0.0
4,USDNOK,50.0
5,USDSGD,0.0
6,USDPLN,50.0
7,USDCAD,50.0
8,USDTRY,0.0
9,EURUSD,100.0


In [11]:
f_estadisticas_ba(mtp,'tabla')

,medida,valor,descripcion
0,Ops totales,74.00,Operaciones totales
1,Ganadoras,41.00,Operaciones ganadoras
2,Ganadoras_c,29.00,Operaciones ganadoras de compra
3,Ganadoras_v,12.00,Operaciones ganadoras de venta
4,Perdedoras,33.00,Operaciones perdedoras
5,Perdedoras_c,16.00,Operaciones perdedoras de compra
6,Perdedoras_v,17.00,Operaciones perdedoras de venta
7,Mediana (profit),12.65,Mediana de profit de operaciones
8,Mediana (pips),13.50,Mediana de pips de operaciones
9,r_efectividad,0.55,Ganadoras totales/ Operaciones totales


In [12]:
f_estadisticas_ba(mtp,'ranking')

,Symbol,rank %
0,EURUSD,54.17
1,USDJPY,83.33
2,GBPUSD,57.14
3,USDCAD,60.00
4,EURGBP,41.67
5,GBPNZD,0.00
6,GBPJPY,100.00
7,CADCHF,0.00
8,EURJPY,0.00
9,EURMXN,100.00


In [13]:
f_estadisticas_ba(mta,'tabla')

,medida,valor,descripcion
0,Ops totales,47.00,Operaciones totales
1,Ganadoras,27.00,Operaciones ganadoras
2,Ganadoras_c,10.00,Operaciones ganadoras de compra
3,Ganadoras_v,17.00,Operaciones ganadoras de venta
4,Perdedoras,20.00,Operaciones perdedoras
5,Perdedoras_c,6.00,Operaciones perdedoras de compra
6,Perdedoras_v,14.00,Operaciones perdedoras de venta
7,Mediana (profit),0.25,Mediana de profit de operaciones
8,Mediana (pips),6.00,Mediana de pips de operaciones
9,r_efectividad,0.57,Ganadoras totales/ Operaciones totales


In [14]:
f_estadisticas_ba(mta,'ranking')

,Symbol,rank %
0,GBPUSD,62.50
1,EURGBP,80.00
2,AUDUSD,42.86
3,EURUSD,50.00
4,GBPJPY,100.00
5,USDCAD,60.00
6,USDJPY,50.00
7,CADJPY,50.00
8,USDCHF,0.00
9,EURJPY,100.00


## MÉTRICAS DE ATRIBUCIÓN AL DESEMPEÑO

In [15]:
def f_evolucion_capital(data):
    prof = data[["Time","Profit"]]
    prof['timestamp'] = pd.to_datetime(data["Time"])
    prof.set_index('timestamp',inplace=True)
    prof = prof.resample("D").sum()
    prof['profit_acm_d'] = 100000 + prof['Profit'].astype(float).cumsum()
    return prof
profsf = f_evolucion_capital(mtf).reset_index()
profsm = f_evolucion_capital(mtm).reset_index()
profsp = f_evolucion_capital(mtp).reset_index()
profsa = f_evolucion_capital(mta).reset_index()

In [23]:
def f_no_habiles(data):
    data = data[(data.timestamp != '2022-09-24') & (data.timestamp != '2022-09-25') & (data.timestamp != '2022-09-17') & (data.timestamp != '2022-09-18')]
    return data

In [24]:
def get_adj_closes(tickers, start_date=None, end_date=None):
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    closes = closes['Adj Close']
    closes.sort_index(inplace=True)
    return closes
benchmarkf = get_adj_closes('SPY', start_date=profsf.iloc[0,0], end_date=profsf.iloc[-1,0])
benchmarkm = get_adj_closes('SPY', start_date=profsm.iloc[0,0], end_date=profsm.iloc[-1,0])
benchmarkp = get_adj_closes('SPY', start_date=profsp.iloc[0,0], end_date=profsp.iloc[-1,0])
benchmarka = get_adj_closes('SPY', start_date=profsa.iloc[0,0], end_date=profsa.iloc[-1,0])

In [25]:
def f_estadisticas_mad(data,benchmark):
        #Sharpe Ratio Original:
        rp = np.mean(np.log(data['profit_acm_d'] / data['profit_acm_d'].shift()).dropna())
        sdp = np.log(data['profit_acm_d'] / data['profit_acm_d'].shift()).dropna().std()
        rf = 0.05/365
        sharp_o = (rp - rf) / sdp
        #Sharpe Ratio Actualizado:
        r_trader = np.log(data['profit_acm_d'] / data['profit_acm_d'].shift()).dropna()
        r_benchmark = np.log(benchmark / benchmark.shift()).dropna()
        rtr = r_trader.tolist()
        rbm = r_benchmark.tolist()
        product = list(map(lambda x,y: x-y ,rtr,rbm))
        sdp2 = st.pstdev(product) 
        sharp_a = (np.mean(r_trader) - np.mean(r_benchmark)) / sdp2
        #DrawDown (Capital):
        draw_down = data['profit_acm_d'].min()
        min_max = data['profit_acm_d'].idxmin()
        fecha_min = data['timestamp'][min_max]
        #DrawUp (Capital):
        draw_up = data['profit_acm_d'].max()
        plus_max = data['profit_acm_d'].idxmax()
        fecha_max = data['timestamp'][plus_max]
        est_mad=pd.DataFrame({'metrica':['sharpe_original','sharpe_actualizado','drawdown_capi','drawdown_capi',
                                         'drawdown_capi','drawup_capi','drawup_capi','drawup_capi'],
                             'valor':[sharp_o, sharp_a, fecha_min, fecha_max ,draw_down, fecha_min, fecha_max ,draw_up],
                             'descripcion':['Sharpe Ratio Fórmula Original','Sharpe Ratio Fórmula Ajustada',
                                            'Fecha inicial del DrawDown de Capital', 'Fecha final del DrawDown de Capital',
                                            'Máxima pérdida flotante registrada', 'Fecha inicial del DrawUp de Capital', 
                                            'Fecha final del DrawUp de Capital','Máxima ganancia flotante registrada']})
        return est_mad

In [26]:
f_estadisticas_mad(f_no_habiles(profsf),benchmarkf)

,metrica,valor,descripcion
0,sharpe_original,-0.510383,Sharpe Ratio Fórmula Original
1,sharpe_actualizado,-0.579393,Sharpe Ratio Fórmula Ajustada
2,drawdown_capi,2022-09-26 00:00:00,Fecha inicial del DrawDown de Capital
3,drawdown_capi,2022-09-22 00:00:00,Fecha final del DrawDown de Capital
4,drawdown_capi,5474.76,Máxima pérdida flotante registrada
5,drawup_capi,2022-09-26 00:00:00,Fecha inicial del DrawUp de Capital
6,drawup_capi,2022-09-22 00:00:00,Fecha final del DrawUp de Capital
7,drawup_capi,97214.9,Máxima ganancia flotante registrada


In [27]:
f_estadisticas_mad(f_no_habiles(profsm),benchmarkm)

,metrica,valor,descripcion
0,sharpe_original,0.171653,Sharpe Ratio Fórmula Original
1,sharpe_actualizado,0.333112,Sharpe Ratio Fórmula Ajustada
2,drawdown_capi,2022-09-19 00:00:00,Fecha inicial del DrawDown de Capital
3,drawdown_capi,2022-09-22 00:00:00,Fecha final del DrawDown de Capital
4,drawdown_capi,96928.8,Máxima pérdida flotante registrada
5,drawup_capi,2022-09-19 00:00:00,Fecha inicial del DrawUp de Capital
6,drawup_capi,2022-09-22 00:00:00,Fecha final del DrawUp de Capital
7,drawup_capi,103285,Máxima ganancia flotante registrada


In [28]:
f_estadisticas_mad(f_no_habiles(profsp),benchmarkp)

,metrica,valor,descripcion
0,sharpe_original,-0.667338,Sharpe Ratio Fórmula Original
1,sharpe_actualizado,-0.408434,Sharpe Ratio Fórmula Ajustada
2,drawdown_capi,2022-09-26 00:00:00,Fecha inicial del DrawDown de Capital
3,drawdown_capi,2022-09-19 00:00:00,Fecha final del DrawDown de Capital
4,drawdown_capi,86731.8,Máxima pérdida flotante registrada
5,drawup_capi,2022-09-26 00:00:00,Fecha inicial del DrawUp de Capital
6,drawup_capi,2022-09-19 00:00:00,Fecha final del DrawUp de Capital
7,drawup_capi,99838.8,Máxima ganancia flotante registrada


In [22]:
#error al sacar la desviación estandar de un solo valor
#f_estadisticas_mad(f_no_habiles(profsa),benchmarka)